In [ ]:
# pip install googletrans
# pip install google-cloud-translate

In [ ]:
import pandas as pd

In [ ]:
%store -r df_extr_text

In [ ]:
df = df_extr_text

In [ ]:
from langdetect import detect, LangDetectException


def detect_language(text):
    try:
        # Detect the language of the text
        return detect(text)
    except LangDetectException:
        # Return a placeholder if language detection fails
        return "unknown"


# Apply the language detection function to the 'extr_text' column
df["language"] = df["extr_text"].apply(detect_language)

# Create a count table for the languages
language_counts = df["language"].value_counts()

# Display the language count table
language_counts

In [ ]:
from nltk.tokenize import sent_tokenize


def chunk_text_by_sentence(text, max_length=512):
    sentences = sent_tokenize(text)
    current_chunk = []
    chunks = []

    for sentence in sentences:
        if len(" ".join(current_chunk) + " " + sentence) > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
        else:
            current_chunk.append(sentence)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [ ]:
def translate(text, target_language="en"):
    url = "https://translation.googleapis.com/language/translate/v2"
    params = {
        "q": text,
        "target": target_language,
        "key": api_key,  # Use the API key from the environment variable
    }
    response = requests.get(url, params=params)
    result = response.json()

    # Check for errors in the response
    if "error" in result:
        print("Error encountered:", result["error"])
        return None

    # Extracting the translated text
    try:
        translated_text = result["data"]["translations"][0]["translatedText"]
        return translated_text
    except KeyError:
        print("Unexpected response format:", result)
        return None


# Example usage
translated_text = translate_text_with_api_key("こんにちは、私の犬はかわいいです", "en")
print(translated_text)

In [ ]:
df_test = pd.DataFrame(
    {
        "ascii_id_company": ["001", "002", "003", "004"],
        "extr_text": [
            "こんにちは、私の犬はかわいいです",
            "这是一段中文文本",
            "여기에 한국어 텍스트가 있습니다",
            "hi baby",
        ],
        "language": ["ja", "zh-cn", "ko", "en"],
    }
)

In [ ]:
# Function to apply on each row of the DataFrame
def translate_row(row):
    text = row["extr_text"]
    src_lang = row["language"]
    # Return None if the text is already in English
    if src_lang == "en":
        return None
    else:
        # If not English, chunk the text
        chunks = chunk_text_by_sentence(text)

        # Translate each chunk
        translated_chunks = [translate(chunk) for chunk in chunks]

        # Combine the translated chunks
        translated_text = " ".join(translated_chunks)

        return translated_text


# Apply the function to each row and create a new column with the translated texts
df_test["translated"] = df_test.apply(translate_row, axis=1)
df_test

In [ ]:
# now test real df

df_test_real = df.head(10)

In [ ]:
df_test_real

In [ ]:
chunk_text_by_sentence(df_test_real.iloc[8, 1])

In [ ]:
df_test_real["translated"] = df_test_real.apply(translate_row, axis=1)
df_test_real